#### Auto-fill Questionnaire using Chain of Thought or Few-Shot Examples

This notebook showcases the application of few-shot examples in autofilling questionnaires. It utilizes a json file (`cot_examples.json`) to 
provide the LLM with example responses for some use-cases.

By leveraging these few-shot examples, we can enable seamless completion of lengthy questionnaires, minimizing manual effort and improving overall efficiency.

In [1]:
from risk_atlas_nexus.blocks.inference import (
    RITSInferenceEngine,
    WMLInferenceEngine,
    OllamaInferenceEngine,
    VLLMInferenceEngine,
)
from risk_atlas_nexus.blocks.inference.params import (
    InferenceEngineCredentials,
    RITSInferenceEngineParams,
    WMLInferenceEngineParams,
    OllamaInferenceEngineParams,
    VLLMInferenceEngineParams,
)

from copy import deepcopy
from risk_atlas_nexus.library import RiskAtlasNexus

/Users/seshu/Documents/2025/fork/risk-atlas-nexus/src/risk_atlas_nexus/toolkit/job_utils.py:2: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


##### Risk Atlas Nexus uses Large Language Models (LLMs) to infer risks dimensions. Therefore requires access to LLMs to inference or call the model. 

**Available Inference Engines**: WML, Ollama, vLLM, RITS. Please follow the [Inference APIs](https://github.com/IBM/risk-atlas-nexus?tab=readme-ov-file#install-for-inference-apis) guide before going ahead.

*Note:* RITS is intended solely for internal IBM use and requires TUNNELALL VPN for access.

In [2]:
# inference_engine = WMLInferenceEngine(
#     model_name_or_path="ibm/granite-20b-code-instruct",
#     credentials={
#         "api_key": "<WML_API_KEY>",
#         "api_url": "<WML_API_URL>",
#         "project_id": "<WML_PROJECT_ID>",
#     },
#     parameters=WMLInferenceEngineParams(
#         max_new_tokens=100, decoding_method="greedy", repetition_penalty=1
#     ),
#     postprocessors=["clean_output"],
# )

inference_engine = OllamaInferenceEngine(
    model_name_or_path="hf.co/ibm-granite/granite-20b-code-instruct-8k-GGUF",
    credentials=InferenceEngineCredentials(api_url="<OLLAMA_API_URL>"),
    parameters=OllamaInferenceEngineParams(
        num_predict=100, num_ctx=8192, temperature=0.7, repeat_penalty=1
    ),
    postprocessors=["clean_output"],
)

# inference_engine = VLLMInferenceEngine(
#     model_name_or_path="ibm-granite/granite-3.1-8b-instruct",
#     credentials=InferenceEngineCredentials(
#         api_url="<VLLM_API_URL>", api_key="<VLLM_API_KEY>"
#     ),
#     parameters=VLLMInferenceEngineParams(max_tokens=100, temperature=0.7),
#     postprocessors=["clean_output"],
# )

# inference_engine = RITSInferenceEngine(
#     model_name_or_path="ibm-granite/granite-3.1-8b-instruct",
#     credentials={
#         "api_key": "<RITS_API_KEY>",
#         "api_url": "<RITS_API_URL>",
#     },
#     parameters=RITSInferenceEngineParams(max_tokens=100),
#     postprocessors=["clean_output"],
# )

[2025-03-05 15:53:16:512] - INFO - RiskAtlasNexus - OLLAMA inference engine will execute requests on the server at localhost:11434.
[2025-03-05 15:53:16:585] - INFO - RiskAtlasNexus - Created OLLAMA inference engine.


##### Create an instance of RiskAtlasNexus

*Note: (Optional)* You can specify your own directory in `RiskAtlasNexus(base_dir=<PATH>)` to utilize custom AI ontologies. If left blank, the system will use the provided AI ontologies.

In [3]:
risk_atlas_nexus = RiskAtlasNexus()

[2025-03-05 15:53:16:644] - INFO - RiskAtlasNexus - YAML ignored: /Users/seshu/Documents/2025/fork/risk-atlas-nexus/src/risk_atlas_nexus/data/knowledge_graph/granite_guardian_dimensions.yaml. Failed to load.
[2025-03-05 15:53:16:648] - INFO - RiskAtlasNexus - Created RiskAtlasNexus instance. Base_dir: None


#### Defining Few-Shot Examples for Auto-Assist Functionality

This cell showcases the template used in `risk_atlas_nexus/data/templates/cot_examples.json` to provide few-shot 
examples for auto-assist functionality.

**Template Structure:**

* Each question is associated with a list of example intents and 
corresponding answers.
* The format is:
	+ `Question`
	+ `- intents:` (list of example intents)
	+ `- answers:` (list of corresponding answers for the respective intents above)

In this notebook, we're using a simplified template to cover 4 questions 
from the Airo questionnaire:

1. System environment
2. Intended domain
3. Utilized techniques
4. Targeted user group

**Customization:**

To adapt this auto-assist functionality to custom questionnaires, users 
need to provide their own set of questions, example intents, and 
corresponding answers in a json file (e.g., `cot_examples.json`). This will enable 
the LLM to learn from these few-shot examples and generate responses for 
unseen queries.

In [4]:
import json
import os
from risk_atlas_nexus.data import get_templates_path

with open(os.path.join(get_templates_path(), "cot_examples.json")) as f:
    cot_data = json.load(f)
print(cot_data)

[{'question': 'In which environment is the system used?', 'examples': {'answers': ["Insurance Claims Processing or Risk Management or Data Analytics \nExplanation: 1. Insurance Claims Processing: The system might be used by an insurance company's claims processing department to analyze and identify patterns in healthcare insurance claims. 2. Risk Management: The system could be applied in risk management teams to detect potential risks and opportunities for cost savings. 3. Data Analytics: The system might be used by a data analytics team within the healthcare insurance company to identify patterns in claims data, helping to inform business decisions.", 'Municipal Waste Management or Private Waste Management departments \nExplanation: Waste management companies need to efficiently collect and process waste while minimizing costs and environmental impact. By using generative AI to analyze historical data on waste generation, recycling rates, and resource utilization, they can optimize c

There are two ways to use the inference engine to get the LLM outputs. `generate_zero_shot_output` which gives the zero-shot output for the question and `generate_few_shot_output` which gives the output using few-shot examples defined above. 

In [5]:
questions = []
for data in deepcopy(cot_data):
    questions.append(data["question"])

usecase = "Generate personalized, relevant responses, recommendations, and summaries of claims for customers to support agents to enhance their interactions with customers."

results = risk_atlas_nexus.generate_zero_shot_output(
    inference_engine, usecase, questions
)

results

Inferring with OLLAMA: 100%|██████████| 7/7 [00:15<00:00,  2.20s/it]


['Based on the given context, the system is likely used in a customer service or sales environment, where agents need to provide personalized and relevant responses, recommendations, and summaries of claims to customers. This suggests that the system is used in a call center, help desk, or sales support setting, where agents interact with customers through phone, email, or chat.',
 "Based on the context of generating personalized, relevant responses, recommendations, and summaries of claims to support agents in their interactions with customers, the system is likely intended to be used in the domain of Customer Service/Support. This domain involves providing assistance and resolving issues for customers, and the system's capabilities align with the needs of customer support teams. The system's ability to generate personalized and relevant responses, recommendations, and summaries of claims would be particularly useful in this context, as it would enable agents to provide",
 'The system

In [6]:
cot_data_with_examples = deepcopy(cot_data)
usecase = "Generate personalized, relevant responses, recommendations, and summaries of claims for customers to support agents to enhance their interactions with customers."

results = risk_atlas_nexus.generate_few_shot_output(
    inference_engine, usecase, cot_data_with_examples
)

results

Inferring with OLLAMA: 100%|██████████| 7/7 [00:36<00:00,  5.27s/it]


['[Given: Generate personalized, relevant responses, recommendations, and summaries of claims for customers to support agents to enhance their interactions with customers.] [Question: In which environment is the system used?] Output: [Insurance Claims Processing or Customer Service or Call Center Operations] Explanation: 1. Insurance Claims Processing: The system might be used by insurance companies to analyze and process claims, providing personalized responses and recommendations to customers. 2. Customer Service: The system could be applied in customer service teams to',
 'Based on the examples provided, I will attempt to categorize the given task into the corresponding domain. Example 1: [Given: Generate personalized, relevant responses, recommendations, and summaries of claims for customers to support agents to enhance their interactions with customers.] [Question: What domain is the system intended to be used in? ...] Output: [Customer service/support] Explanation: Since the task